# 5. SEC Annual Reports Extraction 

Given the GVKEYs and hence the CIK SEC security keys, that is inter-operable with boardex data,

We will then look at the relevant annual reports 

### Instructions

Take the metadata from the first step to get the details for the second step!!

In [5]:
import config

API_KEY = config.CREDS["sec_api"]

In [6]:
from sec_api import QueryApi, ExtractorApi

In [7]:
def concatenate_to_literal_without_quotes(strings: str):
    """
    Useful function to convert to a string literal
    """

    # Convert each string in the list to be surrounded by single quotes
    quoted_strings = [f"{s}" for s in strings]
    # Join the quoted strings with a comma and space
    joined_strings = ' OR '.join(quoted_strings)
    # Format the joined strings within parentheses
    result = f"({joined_strings})"
    return result

In [8]:
import pandas as pd
import numpy as np
import json

In [13]:
sec_df = pd.read_csv("raw_data/sec_cik_gvkey_metadata_us_pharma_company_network_2011_2023.csv", index_col= 0)

In [14]:
sec_df

,cik,gvkey,source,link_desc,sec_company_name,link_company_name,sec_start_date,sec_end_date,link_start_date,link_end_date,...,n10q_a,ndef,n8k,nlet,n13d,n13g,n13f,ntot,ntot_nt,ntot_a
0,318154,1602,CRSP/Compustat Merged,Valid CIK-GVKEY Link,AMGEN INC,AMGEN INC,1994-03-28,2024-06-21,2007-04-14,2024-03-08,...,4.0,92.0,308.0,22.0,31.0,89.0,0.0,2942.0,0.0,182.0
1,318154,1602,Capital IQ,Valid CIK-GVKEY Link,AMGEN INC,AMGEN INC.,1994-03-28,2024-06-21,NaN,NaN,...,4.0,92.0,308.0,22.0,31.0,89.0,0.0,2942.0,0.0,182.0
2,318154,1602,Compustat Company,Valid CIK-GVKEY Link,AMGEN INC,AMGEN INC,1994-03-28,2024-06-21,1983-03-31,2023-12-31,...,4.0,92.0,308.0,22.0,31.0,89.0,0.0,2942.0,0.0,182.0
3,318154,1602,Compustat Security,Valid CIK-GVKEY Link,AMGEN INC,AMGEN INC,1994-03-28,2024-06-21,1999-02-16,2020-02-14,...,4.0,92.0,308.0,22.0,31.0,89.0,0.0,2942.0,0.0,182.0
4,722104,2222,CRSP/Compustat Merged,Valid CIK-GVKEY Link,SAVIENT PHARMACEUTICALS INC,SAVIENT PHARMACEUTICALS INC,1995-04-13,2019-03-20,2007-04-14,2024-03-08,...,10.0,24.0,176.0,13.0,6.0,102.0,0.0,980.0,4.0,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,1671502,351038,CRSP/Compustat Merged,Link with Name Mismatch,"QUOIN PHARMACEUTICALS, LTD.",CELLECT BIOTECHNOLOGY LTD,2016-04-08,2024-06-10,2021-11-13,2023-08-11,...,0.0,4.0,19.0,13.0,3.0,25.0,0.0,419.0,0.0,37.0
4321,1671502,351038,CRSP/Compustat Merged,Valid CIK-GVKEY Link,"QUOIN PHARMACEUTICALS, LTD.",QUOIN PHARMACEUTICALS LTD,2016-04-08,2024-06-10,2023-08-12,2023-11-10,...,0.0,4.0,19.0,13.0,3.0,25.0,0.0,419.0,0.0,37.0
4322,1671502,351038,Capital IQ,Valid CIK-GVKEY Link,"QUOIN PHARMACEUTICALS, LTD.","QUOIN PHARMACEUTICALS, LTD.",2016-04-08,2024-06-10,2021-10-29,NaN,...,0.0,4.0,19.0,13.0,3.0,25.0,0.0,419.0,0.0,37.0
4323,1671502,351038,Compustat Company,Valid CIK-GVKEY Link,"QUOIN PHARMACEUTICALS, LTD.",QUOIN PHARMACEUTICALS LTD,2016-04-08,2024-06-10,2020-12-31,2023-12-31,...,0.0,4.0,19.0,13.0,3.0,25.0,0.0,419.0,0.0,37.0


In [15]:
cik_values = sec_df["cik"].dropna().apply(int).astype(str).drop_duplicates().to_list()

In [16]:
len(cik_values)

1182

API Query

In [9]:
concatenate_to_literal_without_quotes([1,3])

'(1 OR 3)'

In [17]:
cik_query = concatenate_to_literal_without_quotes(cik_values)

In [18]:
cik_query

'(318154 OR 722104 OR 1347178 OR 14272 OR 749647 OR 716646 OR 1302573 OR 109657 OR 727510 OR 320017 OR 38074 OR 1436304 OR 716054 OR 919745 OR 352998 OR 722830 OR 200406 OR 57055 OR 1236271 OR 59478 OR 64978 OR 310158 OR 353569 OR 1906324 OR 69499 OR 1623613 OR 1792044 OR 73290 OR 737207 OR 70793 OR 275053 OR 878088 OR 1142576 OR 1559149 OR 78003 OR 315545 OR 90185 OR 736822 OR 786947 OR 730272 OR 732485 OR 792977 OR 791908 OR 1074404 OR 1245104 OR 794172 OR 725363 OR 318306 OR 715446 OR 728387 OR 801555 OR 1116463 OR 805928 OR 799698 OR 811641 OR 812796 OR 816284 OR 352747 OR 1555280 OR 818033 OR 1505512 OR 16590 OR 820081 OR 820237 OR 3116 OR 1270073 OR 831967 OR 1557939 OR 1557958 OR 1557961 OR 1651347 OR 821995 OR 704562 OR 755806 OR 771856 OR 793171 OR 842023 OR 805326 OR 1293310 OR 1037643 OR 1178711 OR 850693 OR 1551152 OR 1177648 OR 1563665 OR 1438533 OR 1328440 OR 1388320 OR 855654 OR 1498382 OR 834365 OR 1373707 OR 1117480 OR 1477598 OR 1509190 OR 1509261 OR 1368514 OR 146372

1. Basic filing to get a 10K report

Write for loop if greater than 10k filings

https://sec-api.io/docs/query-api

In [19]:
# cik_values

In [20]:
queryApi = QueryApi(api_key=API_KEY)#"YOUR_API_KEY")

In [21]:
# get the full date here , try if this makes sense, and then find the appropraite type below!

# try the full time scale as before, and get everything as needed

query = {
    # "query": f"cik:{cik_query} AND formType:\"10-K\"",
    # "query": "cik:1050122 AND formType:\"10-K\"",
    "query": "cik:318154 AND formType:\"10-K\" AND periodOfReport:[2011-01-01T00:00:00 TO 2024-12-31T23:59:59]",
    # "query": "cik:(1176334 OR 1413898 OR 99780) AND formType:\"10-K\"",
    # "query": "cik:(0001050122) AND formType:\"10-K\"",
    "from": "0",
    "size": "50",
    "sort": [{ "periodOfReport": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)
filings

{'total': {'value': 15, 'relation': 'eq'},
 'query': {'from': 0, 'size': 50},
 'filings': [{'ticker': 'AMGN',
   'formType': '10-K',
   'accessionNo': '0000318154-24-000011',
   'cik': '318154',
   'companyNameLong': 'AMGEN INC (Filer)',
   'companyName': 'AMGEN INC',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/318154/000031815424000011/amgn-20231231.htm',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/318154/000031815424000011/0000318154-24-000011.txt',
   'filedAt': '2024-02-14T16:23:32-05:00',
   'documentFormatFiles': [{'sequence': '1',
     'size': '3754954',
     'documentUrl': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/318154/000031815424000011/amgn-20231231.htm',
     'description': '10-K',
     'type': '10-K'},
    {'sequence': '2',
     'size': '232230',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/318154/000031815424000011/exhibit433-

In [37]:
pd.DataFrame(filings["filings"])

,ticker,formType,accessionNo,cik,companyNameLong,companyName,linkToFilingDetails,description,linkToTxt,filedAt,documentFormatFiles,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles,effectivenessDate
0,AMGN,10-K,0000318154-24-000011,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2024-02-14T16:23:32-05:00,"[{'sequence': '1', 'size': '3754954', 'documen...",2023-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",c192751c88f50d8ca603ab72c5840583,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '17', 'size': '118693', 'documen...",NaN
1,AMGN,10-K,0000318154-23-000017,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2023-02-09T16:26:31-05:00,"[{'sequence': '1', 'size': '4409594', 'documen...",2022-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",3f8f2f776c43180b8573405b00605c15,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '11', 'size': '114316', 'documen...",NaN
2,AMGN,10-K,0000318154-22-000010,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2022-02-16T16:39:53-05:00,"[{'sequence': '1', 'size': '4325355', 'documen...",2021-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",1d205d8c366bf6e51746f33967a1141d,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '13', 'size': '109735', 'documen...",NaN
3,AMGN,10-K,0000318154-21-000010,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2021-02-08T19:24:56-05:00,"[{'sequence': '1', 'size': '4446756', 'documen...",2020-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",6de209e21fcaed407f67723b8296595f,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '11', 'size': '111994', 'documen...",2020-12-31
4,AMGN,10-K/A,0000318154-20-000019,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K/A - Annual report [Section 13 and 15...,https://www.sec.gov/Archives/edgar/data/318154...,2020-02-13T13:51:00-05:00,"[{'sequence': '1', 'size': '58920', 'documentU...",2019-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",c1d1980920a0c0097672d1b5f3f8185f,[],https://www.sec.gov/Archives/edgar/data/318154...,,[],NaN
5,AMGN,10-K,0000318154-20-000017,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2020-02-12T17:15:19-05:00,"[{'sequence': '1', 'size': '4740655', 'documen...",2019-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",de969a0e8e1a4e79639e4b5decd373ba,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '18', 'size': '110005', 'documen...",NaN
6,AMGN,10-K,0000318154-19-000008,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2019-02-13T17:20:20-05:00,"[{'sequence': '1', 'size': '3452299', 'documen...",2018-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",b3d048fef614f6bb746e58ccd4bdfa4c,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '8', 'size': '4702165', 'documen...",NaN
7,AMGN,10-K,0000318154-18-000004,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2018-02-13T16:48:04-05:00,"[{'sequence': '1', 'size': '3444349'

# Getting all the data we need by iterating

https://sec-api.io/docs/sec-filings-render-api/python-example

Running things in parallel

In [65]:
from pandarallel import pandarallel
import pandas as pd
import json
import os

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Iterate over everything we need

In [52]:
def get_all_filings(cik:str):
    """
    Get the filings by CIK, specify date below
    """

    query = {
        "query": f"cik:{cik} AND formType:\"10-K\" AND periodOfReport:[2011-01-01T00:00:00 TO 2023-12-31T23:59:59]",
        "from": "0",
        "size": "50",
        "sort": [{ "periodOfReport": { "order": "desc" } }]
    }

    all_filings = []
    from_value = 0
    size_value = int(query['size'])

    while True:
        query['from'] = str(from_value)
        filings = queryApi.get_filings(query)
        
        # Add current batch of filings to the all_filings list
        all_filings.extend(filings['filings'])
        
        # Check if the relation is not 'gte', break the loop
        if filings['total']['relation'] != 'gte':
            break
        
        # Increment the from value by size for the next iteration
        from_value += size_value

    return all_filings

In [53]:
# all_filings = get_all_filings("318154")
# len(all_filings)

##### Get all relevant filings in parallel below

In [54]:
def save_to_jsonl(filename, data):
    """
    Append a list of dictionaries to a JSONL file.
    """
    with open(filename, 'a') as f:
        for entry in data:
            json.dump(entry, f)
            f.write('\n')

In [75]:
def process_and_save_cik(cik):
    """
    Get all filings for a given CIK and save them to a temporary JSONL file.
    """
    filings = get_all_filings(cik)
    temp_filename = f'temp/temp_{cik}.jsonl'
    save_to_jsonl(temp_filename, filings)

In [76]:
def combine_temp_files(temp_files, final_filename):
    """
    Combine multiple JSONL files into one.
    """
    with open(final_filename, 'a') as final_file:
        for temp_file in temp_files:
            with open(temp_file, 'r') as tf:
                for line in tf:
                    final_file.write(line)
            os.remove(temp_file)

In [77]:
def read_jsonl_range(file_path, start_line, end_line):
    # Container for selected data
    data = []

    # Open and iterate over the lines of the file
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            # Check if the current line is within the desired range
            if start_line <= i < end_line:
                # Parse JSON data from the line and add to the list
                data.append(json.loads(line))
            elif i >= end_line:
                # Stop reading if the end line is reached
                break

    # Create a DataFrame from the list of dictionaries
    return pd.DataFrame(data)


### Run parallelised workflow

Convert everything to dataframe and run apply to save everything to jsonl

In [78]:
len(cik_values), cik_values[:5]

(1182, ['318154', '722104', '1347178', '14272', '749647'])

In [79]:
cik_series = pd.Series(cik_values)

In [80]:
cik_series.parallel_apply(process_and_save_cik)

0       None
1       None
2       None
3       None
4       None
        ... 
1177    None
1178    None
1179    None
1180    None
1181    None
Length: 1182, dtype: object

In [94]:
FILE_EXPORT_NAME = "raw_data/sec_us_phrama_all_company_filing_metadata_references_2011_2023.jsonl"

temp_files = [f'temp/temp_{cik}.jsonl' for cik in cik_values]

combine_temp_files(temp_files, FILE_EXPORT_NAME)

SyntaxError: invalid syntax (1181281449.py, line 7)

Changed the report date above accordingly to only get 3 years's worth of annual reports.

2. Then get the url for the special section for the report that's needed

Once you get the dictionary then save the json, and then find the json details and call the corresponding api that follows

Here we search the text as following

https://www.sec.gov/edgar/search/

Here using CIK code: '318154'

In [83]:
from sec_api import ExtractorApi

extractorApi = ExtractorApi(API_KEY)

Get the report data, look at the relevant report sections below

Particularly the financial report sections

In [87]:
sec_annual_report_meta_df = pd.read_json("raw_data/sec_us_phrama_all_company_filing_metadata_references_2011_2023.jsonl", lines = True)
sec_annual_report_meta_df.head()

,ticker,formType,accessionNo,cik,companyNameLong,companyName,linkToFilingDetails,description,linkToTxt,filedAt,documentFormatFiles,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles,effectivenessDate
0,AMGN,10-K,0000318154-24-000011,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2024-02-14T16:23:32-05:00,"[{'sequence': '1', 'size': '3754954', 'documen...",2023-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",c192751c88f50d8ca603ab72c5840583,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '17', 'size': '118693', 'documen...",NaN
1,AMGN,10-K,0000318154-23-000017,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2023-02-09T16:26:31-05:00,"[{'sequence': '1', 'size': '4409594', 'documen...",2022-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",3f8f2f776c43180b8573405b00605c15,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '11', 'size': '114316', 'documen...",NaN
2,AMGN,10-K,0000318154-22-000010,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2022-02-16T16:39:53-05:00,"[{'sequence': '1', 'size': '4325355', 'documen...",2021-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",1d205d8c366bf6e51746f33967a1141d,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '13', 'size': '109735', 'documen...",NaN
3,AMGN,10-K,0000318154-21-000010,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2021-02-08T19:24:56-05:00,"[{'sequence': '1', 'size': '4446756', 'documen...",2020-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",6de209e21fcaed407f67723b8296595f,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '11', 'size': '111994', 'documen...",2020-12-31
4,AMGN,10-K/A,0000318154-20-000019,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K/A - Annual report [Section 13 and 15...,https://www.sec.gov/Archives/edgar/data/318154...,2020-02-13T13:51:00-05:00,"[{'sequence': '1', 'size': '58920', 'documentU...",2019-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",c1d1980920a0c0097672d1b5f3f8185f,[],https://www.sec.gov/Archives/edgar/data/318154...,,[],NaN


First we get the report urls for there reports.

In [88]:
report_urls = sec_annual_report_meta_df["linkToFilingDetails"]

In [50]:
len(all_filings_all_dict.keys())

781

In [91]:
report_url = report_urls.iloc[0]

In [92]:
section_text = extractorApi.get_section(report_url, "1", "text")
print(section_text)

 Item 1. BUSINESS ##TABLE_END

Amgen Inc. (including its subsidiaries, referred to as &#8220;Amgen,&#8221; &#8220;the Company,&#8221; &#8220;we,&#8221; &#8220;our&#8221; or &#8220;us&#8221;) discovers, develops, manufactures and delivers innovative medicines to fight some of the world&#8217;s toughest diseases. Amgen focuses on areas of high unmet medical need and leverages its expertise to strive for solutions that dramatically improve people&#8217;s lives, while also reducing the social and economic burden of disease. We helped launch the biotechnology industry more than 40 years ago and have grown to be one of the world&#8217;s leading independent biotechnology companies. Our robust pipeline includes potential first-in-class medicines at all stages of development. We have a presence in approximately 100 countries worldwide. 

Amgen was incorporated in California in 1980 and became a Delaware corporation in 1987. Amgen operates in one business segment: human therapeutics. 

Significa

In [93]:
risk_section_text = extractorApi.get_section(report_url, "1A", "text")
print(risk_section_text)

 Item 1A. RISK FACTORS ##TABLE_END

This report and other documents we file with the SEC contain forward-looking statements that are based on current expectations, estimates, forecasts and projections about us, our future performance, our business, our beliefs and our management&#8217;s assumptions. These statements are not guarantees of future performance and involve certain risks, uncertainties and assumptions that are difficult to predict. You should carefully consider the risks and uncertainties our business faces. The risks described below are not the only ones we face. Our business is also subject to the risks that affect many other companies, such as employment relations, general economic conditions, geopolitical events and international operations. Further, additional risks not currently known to us or that we currently believe are immaterial may in the future materially and adversely affect our business, operations, liquidity and stock price. 

SUMMARY 

Risks Related to Gover

In [96]:
# check the other section which is of relevance!!

print(extractorApi.get_section(report_url, "7", "text"))

 Item 7. MANAGEMENT&#8217;S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS ##TABLE_END

The following MD&#38;A is intended to assist the reader in understanding Amgen&#8217;s business. MD&#38;A is provided as a supplement to, and should be read in conjunction with, our consolidated financial statements and accompanying notes. Our results of operations discussed in MD&#38;A are presented in conformity with GAAP. Amgen operates in one business segment: human therapeutics. Therefore, our results of operations are discussed on a consolidated basis. 

Forward-looking statements 

This report and other documents we file with the SEC contain forward-looking statements that are based on current expectations, estimates, forecasts and projections about us, our future performance, our business, our beliefs and our management&#8217;s assumptions. In addition, we, or others on our behalf, may make forward-looking statements in press releases, written statements or our comm

# IMPORTANT - RUN FROM HERE IF ALREADY HAVE THE METADATA!!


## Get all of the annual report text

Iteratively call all of the reports in the last 3 years, simplified the workflow - so this is fine.

Since dataframe structure, break it down iteratively and use batches as the generators here instead!!

In [113]:
# from tqdm import tqdm

In [119]:
# # iterate each item for dictionary

# all_filings_all_lst_with_text = []

# for filing_dict in tqdm(all_filings_all_lst):

#     report_url = filing_dict["linkToFilingDetails"]

#     filing_dict["Text_1"] = extractorApi.get_section(report_url, "1", "text")
#     filing_dict["Text_1A"] = extractorApi.get_section(report_url, "1A", "text")

#     all_filings_all_lst_with_text.append(filing_dict)

In [124]:
sec_annual_report_meta_df = pd.read_json("raw_data/sec_us_phrama_all_company_filing_metadata_references_2011_2023.jsonl", lines = True)
sec_annual_report_meta_df.head(2)

,ticker,formType,accessionNo,cik,companyNameLong,companyName,linkToFilingDetails,description,linkToTxt,filedAt,documentFormatFiles,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles,effectivenessDate
0,AMGN,10-K,0000318154-24-000011,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2024-02-14T16:23:32-05:00,"[{'sequence': '1', 'size': '3754954', 'documen...",2023-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",c192751c88f50d8ca603ab72c5840583,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '17', 'size': '118693', 'documen...",NaN
1,AMGN,10-K,0000318154-23-000017,318154,AMGEN INC (Filer),AMGEN INC,https://www.sec.gov/Archives/edgar/data/318154...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/318154...,2023-02-09T16:26:31-05:00,"[{'sequence': '1', 'size': '4409594', 'documen...",2022-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",3f8f2f776c43180b8573405b00605c15,[],https://www.sec.gov/Archives/edgar/data/318154...,,"[{'sequence': '11', 'size': '114316', 'documen...",NaN


In [125]:
# save all in ram - should be enough, probably time is a bit longer

def add_factors(row):
    report_url = row["linkToFilingDetails"]

    try:

        # row["Text_1"] = extractorApi.get_section(report_url, "1", "text")
        # row["Text_1A"] = extractorApi.get_section(report_url, "1A", "text")
        row["Text_7"] = extractorApi.get_section(report_url, "7", "text")
        row["Text_7A"] = extractorApi.get_section(report_url, "7A", "text")
    except:
        # row["Text_1"] = None
        # row["Text_1A"] = None
        row["Text_7"] = None
        row["Text_7A"] = None


    
    # Add more factors as needed
    return row

In [126]:
# Apply the add_factors function to each row in parallel, separate files if there is a ram issue later.
sec_annual_report_with_text = sec_annual_report_meta_df.parallel_apply(add_factors, axis=1)

## Retry the above approach for other sections

If the above doesn't work too well. 30 minutes etc. is correct, and edit it save the results, works fine - with enough ram to work through everything - so can run this again. Just need to parallel everything for the preprocessing and the LDA workflow!!

In [130]:
sec_annual_report_with_text_filtered = sec_annual_report_with_text[sec_annual_report_with_text["Text_7"].eq(None) | sec_annual_report_with_text["Text_7A"].eq(None)]
sec_annual_report_with_text_filtered

,ticker,formType,accessionNo,cik,companyNameLong,companyName,linkToFilingDetails,description,linkToTxt,filedAt,...,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles,effectivenessDate,Text_7,Text_7A


In [128]:
# sec_annual_report_with_text[sec_annual_report_with_text["Text_1"]!= None]

Please Note if things fail then just run it again!!

In [129]:
# Save the processed DataFrame to a new JSONL file
sec_annual_report_with_text.to_json('raw_data/sec_us_phrama_all_company_filing_meta_with_text_w_7_7A_2011_2023.jsonl', orient='records', lines=True) # fast write up anyways

Fixed - with jsonl (and not adding references - file size is small - so LDA should work fine as well).

So LDA runtime should be fast as well. Try to improve efficiency of this for finetuning - see how long it would take for the new notebook. Do that in the evening!!

### Next steps

Do LDA on the preprocessing for the topics and see if there are any differences.

**Can also get other report sections as needed - as seen in the other reports**

Additional preprocessing is needed - after having read the documnents.

They're fairly similar words, so need to remove them as stop words. Look at the other relevant GitHub projects to help with all this.

Files are still large - but can still run it on Google Colab try to do that later.

As seen on the PhD 10K code - they got it running on Google Colab so do that correctly!!.